In [ ]:
%pip install spacy_transformers

In [ ]:
%pip install -U spacy

In [3]:
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm

In [4]:
spacy.__version__

'3.6.1'

In [28]:
# Load the annotated data from a JSON file
import json
cv_data = json.load(open(r"CV-Parsing-using-Spacy-3-master\data\training\train_data.json"))

In [29]:
len(cv_data)

200

In [41]:
cv_data[1]

["Harini Komaravelli Test Analyst at Oracle, Hyderabad  Hyderabad, Telangana - Email me on Indeed: indeed.com/r/Harini- Komaravelli/2659eee82e435d1b  ➢ 6 Yrs. of IT Experience in Manual and Automation testing.  WORK EXPERIENCE  QA Analyst  Oracle  Test Analyst at Oracle, Hyderabad  Infosys Ltd -  Hyderabad, Telangana -  November 2011 to February 2016  Hyderabad from Nov 2011 to Feb17 2016 ➢ Worked in Tata Consultancy Services, Hyderabad from Feb 24 to Apr 11 2017 ➢ Currently working as a Test Analyst at Oracle, Hyderabad  QA Analyst with 6 years of IT experience  Oracle  EDUCATION  MCA  Osmania University  B.Sc. in Computer Science  Osmania University  SKILLS  Functional Testing, Blue Prism, Qtp  ADDITIONAL INFORMATION  Area of Expertise:  ➢ Familiar with Agile Methodologies. ➢ Having knowledge in Energy (Petroleum) & Health Care domains. ➢ Involved in preparation of Test Scenarios. ➢ Preparing Test Data for the test cases.  https://www.indeed.com/r/Harini-Komaravelli/2659eee82e435d1b?

In [26]:
!python -m spacy init fill-config CV-Parsing-using-Spacy-3-master/data/training/base_config.cfg CV-Parsing-using-Spacy-3-master/data/training/config.cfg

✔ Auto-filled config with all values
✔ Saved config
CV-Parsing-using-Spacy-3-master\data\training\config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [36]:
# Define a function to create spaCy DocBin objects from the annotated data
def get_spacy_doc(file, data):
  # Create a blank spaCy pipeline
  nlp = spacy.blank('en')
  db = DocBin()

  # Iterate through the data
  for text, annot in tqdm(data):
    doc = nlp.make_doc(text)
    annot = annot['entities']

    ents = []
    entity_indices = []

    # Extract entities from the annotations
    for start, end, label in annot:
      skip_entity = False
      for idx in range(start, end):
        if idx in entity_indices:
          skip_entity = True   # if entities are overlapping
          break
      if skip_entity:
        continue

      entity_indices = entity_indices + list(range(start, end))

      try:
        span = doc.char_span(start, end, label=label, alignment_mode='strict')
      except:
        continue

      if span is None:
        # Log errors for annotations that couldn't be processed
        err_data = str([start, end]) + "    " + str(text) + "\n"
        file.write(err_data)
      else:
        ents.append(span)

    try:
        doc.ents = ents
        db.add(doc)
    except:
        pass

  return db


In [ ]:
%pip install scikit-learn

In [37]:
# Split the annotated data into training and testing sets
from sklearn.model_selection import train_test_split
train, test = train_test_split(cv_data, test_size=0.3)

In [38]:
# Display the number of items in the training and testing sets
len(train), len(test)

(140, 60)

In [43]:
# Open a file to log errors during annotation processing
file = open('CV-Parsing-using-Spacy-3-master/data/model_data/error_file.txt','w', encoding='utf-8')

# Create spaCy DocBin objects for training and testing data
db = get_spacy_doc(file, train)
db.to_disk('CV-Parsing-using-Spacy-3-master/data/model_data/train_data.spacy')

db = get_spacy_doc(file, test)
db.to_disk('CV-Parsing-using-Spacy-3-master/data/model_data/test_data.spacy')

# Close the error log file
file.close()

100%|██████████| 60/60 [00:00<00:00, 67.47it/s]


In [44]:
len(db.tokens)

60

In [ ]:
!python -m spacy train CV-Parsing-using-Spacy-3-master/data/training/config.cfg --output CV-Parsing-using-Spacy-3-master/data/model_output --paths.train CV-Parsing-using-Spacy-3-master/data/model_data/train_data.spacy --paths.dev CV-Parsing-using-Spacy-3-master/data/model_data/test_data.spacy --gpu-id 0